In [1]:
def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))

def get_region(matrix, coordinates):
    area = []
    fx, fy = coordinates
    
    plant = matrix[fy][fx]

    def survey(coordinates):
        x,y = coordinates
        region = [(x, y)]
        area.append((x, y))
    
        if x > 0:
            if matrix[y][x-1] == plant:
                next_area = (x-1, y)
                if next_area not in area:
                    # print("matrix[y][x-1]")
                    region.extend(survey(next_area))
            
        if x < len(matrix[y])-1:
            if matrix[y][x+1] == plant:
                next_area = (x+1, y)
                if next_area not in area:
                    # print("matrix[y][x+1]")
                    region.extend(survey(next_area))
    
        if y > 0:
            if matrix[y-1][x] == plant:
                next_area = (x, y-1)
                if next_area not in area:
                    # print("matrix[y-1][x]")
                    region.extend(survey(next_area))
    
        if y < len(matrix)-1:
            if matrix[y+1][x] == plant:
                next_area = (x, y+1)
                if next_area not in area:
                    # print("matrix[y+1][x]")
                    region.extend(survey(next_area))

        return region

    return survey(coordinates)


def get_perimeter(region: list[int]):
    perimeter = 0
    for x,y in region:
        count = 4
        edges = [
            (x-1, y),
            (x+1, y),
            (x, y-1),
            (x, y+1)
        ]
        
        for edge in edges:
            if edge in region:
                count -= 1

        perimeter += count
    return perimeter

def get_regions(matrix: list[list[str]]):
    seen = []
    regions = []
    
    for y in range(len(matrix)):
        for x in range(len(matrix[y])):
            if (x,y) in seen:
                continue
            region = get_region(matrix, (x,y))
            seen.extend(region)
            regions.append(region)

    return regions

with open("input.txt", "r") as f:
    garden = f.read().strip()

matrix = get_matrix(garden)

regions = get_regions(matrix)

out = 0
for region in regions:
    perimeter = get_perimeter(region)
    out += len(region) * get_perimeter(region)

out

1451030

Correct: `1451030`

In [3]:
from itertools import cycle
from colorama import Fore

def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])

def get_x_axis(region, y):
    return [point[0] for point in filter(lambda point: point[1] == y, region)]

def get_y_axis(region, x):
    return [point[1] for point in filter(lambda point: point[0] == x, region)]

def get_sides(region):

    def get_horizontal(region, y, diff):
        x_axis = get_x_axis(region, y)
        sides = 0
        side_present = False
        for x in range(min(x_axis)-1, max(x_axis)+2):
            if (x,y) in region and (x,diff) not in region:
                side_present = True
                continue

            if (x,y) in region and (x,diff) in region:
                if side_present:
                    sides += 1
                    side_present = False
                continue

            if (x,y) not in region:
                if side_present:
                    sides += 1
                    side_present = False
        return sides


    def get_vertical(region, x, diff):
        y_axis = get_y_axis(region, x)
        sides = 0
        side_present = False

        for y in range(min(y_axis)-1, max(y_axis)+2):
            if (x,y) in region and (diff,y) not in region:
                side_present = True
                continue

            if (x,y) in region and (diff,y) in region:
                if side_present:
                    sides += 1
                    side_present = False
                continue

            if (x,y) not in region:
                if side_present:
                    sides += 1
                    side_present = False
        return sides

    x_values, y_values = zip(*region)
    sides = 0
    for y in range(min(y_values), max(y_values)+1):
        x_top = get_horizontal(region, y, y-1)
        x_bottom = get_horizontal(region, y, y+1)
        sides += (x_top + x_bottom)

    for x in range(min(x_values), max(x_values)+1):
        y_left = get_vertical(region, x, x-1)
        y_right = get_vertical(region, x, x+1)
        sides += (y_left + y_right)
    
    return sides


with open("input.txt", "r") as f:
    garden = f.read().strip()

matrix = get_matrix(garden)
regions = get_regions(matrix)
colors = cycle([Fore.GREEN, Fore.RED, Fore.YELLOW, Fore.BLUE, Fore.MAGENTA, Fore.CYAN,  Fore.BLACK])

price = 0
for i, region in enumerate(regions):
    color = next(colors)
    x,y = region[0]
    char = matrix[y][x]
    
    for x,y in region: # get_outer_blocks(region):
        matrix[y][x] = color + matrix[y][x] + Fore.RESET

    sides = get_sides(region)

    # print(f"{char=} {outer_sides=} {inner_sides=} {len(region)=}")
    price += sides * len(region)

print()
print(f"{price=}")
print()
# print(render_matrix(matrix))


price=859494



# Attempts

- `836380` That's not the right answer; your answer is too low.
- `859494` **Correct!**
- `899958` That's not the right answer; your answer is too high.
- `919798` 
- `2317362` That's not the right answer; your answer is too high.